In [2]:
import json
from pathlib import Path

import numpy as np
import torch as t
import webdataset as wds

from thought_transfer.sparse_recoder import SparseRecoder

In [4]:
model_id = "v1_sae1_100000_20240830231731"
config = json.loads(Path(f"../outputs/models/{model_id}.json").read_text())
model = SparseRecoder(**config)
model.load_state_dict(t.load(f"../outputs/models/{model_id}.pth"))
model = model.cuda()

In [5]:
values = np.random.rand(5, 768)
values = t.tensor(values, dtype=t.float32).cuda()
enc, recons = model(values)
print(enc.shape, recons.shape)
print(t.mean((values - recons) ** 2))

torch.Size([5, 6144]) torch.Size([5, 768])
tensor(19.3935, device='cuda:0', grad_fn=<MeanBackward0>)


In [10]:
from thought_transfer.train import unpack

/claire-rcp-scratch/home/terekhov/mats/thought-transfer/src/thought_transfer


In [ ]:
data = 